In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
!pip install unidecode

     |████████████████████████████████| 241 kB 32.8 MB/s 


In [7]:
!pip install word2number

  Created wheel for word2number: filename=word2number-1.1-py3-none-any.whl size=5581 sha256=f103e5d17cc5b61140ba329b9e49de7781a3c11a1b30730b1f93a07f6b53ea1e
  Stored in directory: /root/.cache/pip/wheels/4b/c3/77/a5f48aeb0d3efb7cd5ad61cbd3da30bbf9ffc9662b07c9f879
Successfully built word2number


In [9]:
!pip install contractions

     |████████████████████████████████| 321 kB 31.9 MB/s 
     |████████████████████████████████| 283 kB 26.3 MB/s 
  Created wheel for pyahocorasick: filename=pyahocorasick-1.4.2-cp37-cp37m-linux_x86_64.whl size=85450 sha256=3238c279acf39250bbb924fe478ba64e52234087139f4194edf359df17b16c1b
  Stored in directory: /root/.cache/pip/wheels/25/19/a6/8f363d9939162782bb8439d886469756271abc01f76fbd790f
Successfully built pyahocorasick


In [10]:
from bs4 import BeautifulSoup
import spacy
import unidecode
from word2number import w2n
import contractions

In [13]:
nlp = spacy.load('en_core_web_sm')

In [39]:
# Removing HTML tags from data
def strip_html_tags(text):
  soup = BeautifulSoup(text, "html.parser")
  stripped_text = soup.get_text(separator= " ")
  return stripped_text

In [15]:
# Removing accent from letters
def remove_accented_chars(text):
  text = unidecode.unidecode(text)
  return text

In [16]:
# Expanding Contractions
def expand_contractions(text):
  text = contractions.fix(text)
  return text

In [30]:
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [35]:
# Removing extra whitespaces
def remove_whitespace(text):
    """remove extra whitespaces from text"""
    text = text.strip()
    return " ".join(text.split())

In [36]:
def text_preprocessing(text, accented_chars=True, contractions=True, 
                       convert_num=True, extra_whitespace=True, 
                       lemmatization=True, lowercase=True, punctuations=True,
                       remove_html=True, remove_num=True, special_chars=True, 
                       stop_words=True):
    """preprocess text with default option set to true for all steps"""
    if remove_html == True: #remove html tags
        text = strip_html_tags(text)
    if extra_whitespace == True: #remove extra whitespaces
        text = remove_whitespace(text)
    if accented_chars == True: #remove accented characters
        text = remove_accented_chars(text)
    if contractions == True: #expand contractions
        text = expand_contractions(text)
    if lowercase == True: #convert all characters to lowercase
        text = text.lower()

    doc = nlp(text) #tokenise text

    clean_text = []
    
    for token in doc:
        flag = True
        edit = token.text
        # remove stop words
        if stop_words == True and token.is_stop and token.pos_ != 'NUM': 
            flag = False
        # remove punctuations
        if punctuations == True and token.pos_ == 'PUNCT' and flag == True: 
            flag = False
        # remove special characters
        if special_chars == True and token.pos_ == 'SYM' and flag == True: 
            flag = False
        # remove numbers
        if remove_num == True and (token.pos_ == 'NUM' or token.text.isnumeric()) \
        and flag == True:
            flag = False
        # convert number words to numeric numbers
        if convert_num == True and token.pos_ == 'NUM' and flag == True:
            edit = w2n.word_to_num(token.text)
        # convert tokens to base form
        elif lemmatization == True and token.lemma_ != "-PRON-" and flag == True:
            edit = token.lemma_
        # append tokens edited and not removed to list 
        if edit != "" and flag == True:
            clean_text.append(edit)        
    return clean_text

In [42]:
text = """I'd like to have three cups of coffee<br /><br />from your Café. #delicious"""

In [45]:
clean_text = text_preprocessing(text)
clean_text

['like', 'cup', 'coffee', 'cafe', '#', 'delicious']